<a href="https://colab.research.google.com/github/fer-aguirre/hackathon-somos-nlp-2023/blob/main/colab/sentimiento_tweets_gpt3_fewshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💡 **Introducción a Argilla**


> Argilla es una plataforma de código abierto para la anotación de datos e incorporación de feedback humano en proyectos de PLN.


Está compuesta de dos partes:

1. **Librería Python para manejar datos y integrar con procesos de ML** (predicción, evaluación, active learning, reentrenamiento, etc).

2. **Interfaz de usuario** para analizar y etiquetar datos.


Demo: https://huggingface.co/spaces/argilla/live-demo

AutoTrain, No-code data manager y más: https://huggingface.co/spaces/argilla/argilla-streamlit-customs 

Documentación: https://docs.argilla.io/en/latest/



## **Desplegar Argilla**

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:


**Deploy Argilla on Hugging Face Spaces**: If you want to run tutorials with external notebooks (e.g., Google Colab) and you have an account on Hugging Face, you can deploy Argilla on Spaces with a few clicks:

[![deploy on spaces](https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg)](https://huggingface.co/new-space?template=argilla/argilla-template-space)

For details about configuring your deployment, check the [official Hugging Face Hub guide](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla).


**Launch Argilla using Argilla's quickstart Docker image**: This is the recommended option if you want [Argilla running on your local machine](../../getting_started/quickstart.ipynb). Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip
    
This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

## **Setup**


In [ ]:
%pip install openai datasets argilla sentence-transformers==2.2.2 setfit==0.6.0 -qqq

Importamos `argilla` para manejar datos, predicciones y anotaciones desde Python.

In [ ]:
import argilla as rg

Si estamos usando una instancia remota, en Hugging Face por ejemplo, debemos configurar `API_URL` y `API_KEY` para comunicarnos con la instancia de Argilla:

In [ ]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
rg.init(
    api_url="https://dvilasuero-taller-somosnlp.hf.space", 
    api_key="team.apikey"
)

Ahora incluimos todos los módulos y métodos que necesitamos:

In [ ]:
import os
from json import loads

import openai

from datasets import load_dataset

import pandas as pd

from argilla.metrics.text_classification import f1
from argilla.metrics.commons import text_length

from setfit import get_templated_dataset
from setfit import SetFitModel, SetFitTrainer

from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import CosineSimilarityLoss

## **Leer el dataset**

Vamos a utilizar un dataset del Hub de Hugging Face con tweets

In [ ]:
dataset = load_dataset("pysentimiento/es_sentiment")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# preview dataset content
dataset["train"].to_pandas().head(15)

,text,lang,label
0,@morbosaborealis jajajaja... eso es verdad... ...,es,0
1,@Adriansoler espero y deseo que el interior te...,es,1
2,"comprendo que te molen mis tattoos, pero no te...",es,1
3,"Mi última partida jugada, con Sona support. La...",es,2
4,Tranquilos que con el.dinero de Camacho seguro...,es,2
5,"@daniacal aún no, pero si estará jugable en el...",es,2
6,@ragnomuelle Yo a veces hecho de menos mi pelo...,es,0
7,A mí nunca me podrán hacer una broma porque no...,es,1
8,#feliz septiembre..es bonito retarse..es incre...,es,2
9,Este año el Madrid hará triplete y si lo hace ...,es,2


## **Clasificador de sentimiento con GPT-3**

En esta sección, vamos a utilizar el modelo `text-davinci-003` para clasificar el sentimiento de tweets en Español.



![Screenshot of Argilla UI](https://github.com/argilla-io/argilla/blob/main/docs/_source/_static/tutorials/labelling-textclassification-gpt3-fewshot/labelling-textclassification-gpt3-fewshot-2.png?raw=1)

La estructura principal de nuestro prompt es:

1. Definir la tarea: clasificación de sentimientos de las solicitudes de los clientes
2. Definir el formato y las etiquetas: queremos tres etiquetas y un formato JSON (hasta ahora, este formato solo funcionó para text-davinci-003)
3. Definir el texto a clasificar: esta parte la agregaremos para cada ejemplo en nuestro conjunto de datos.

Probablemente lo más interesante de este prompt es que **pedimos al modelo que explique su predicción y la añada a la respuesta**. Verás más adelante que este es un mecanismo poderoso para entender las decisiones del modelo, la tarea e incluso revisar nuestra verdad básica etiquetada manualmente.

Como queremos probar las capacidades de zero-shot, no proporcionaremos ningún ejemplo. En futuros tutoriales, ampliaremos esto con N-shot proporcionando ejemplos en el mensaje en sí, y también mostraremos cómo se puede ajustar GPT-3 con ejemplos etiquetados. Si estás interesado, ¡únete a nuestra comunidad y hablemos!

A continuación, definimos la plantilla de mensaje, a la cual agregaremos el texto a clasificar antes de llamar a la función openai.Completion.create.

In [ ]:
PROMPT_TEMPLATE = """
Classify the sentiment of the tweet in Spanish using the following JSON format. Use positive, negative, and neutral in lowercase:

{"prediction": sentiment label string, "explanation": sentence string describing in Spanish why you think is the sentiment}

Tweet: 

"""

Ahora, definamos nuestra función de clasificación. Esta función agrega el texto de entrada a la plantilla de mensaje, llama a la API de OpenAI e intenta analizar la respuesta JSON. En algunos de nuestros experimentos, a veces el JSON devuelto no es válido. Tenemos esto en cuenta y marcamos esas predicciones como None y agregamos la respuesta JSON en el campo de explicación.

In [ ]:
# set your api key as ENV, for example with Python: os.environ["OPENAI_API_KEY"] = "your api key"
openai.api_key = os.getenv("OPENAI_API_KEY") 

def classify(text):
    # build prompt with template and input
    prompt = f"{PROMPT_TEMPLATE}\n{text}\n"
    # use create completion template
    completion = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0,
      max_tokens=256
    )
    # get first choice text
    json_response = completion["choices"][0]["text"].strip()
    try:
        prediction = loads(json_response)
    except:
        # for some examples, json is not correctly formatted
        return {"prediction": None, "explanation": f"Wrong JSON format: {json_response}" }
    return prediction  

Ahora llamemos a este método para cada ejemplo en nuestro conjunto de prueba de sentimiento bancario para que podamos compararlo con otros métodos (SetFit, GPT-3 de pocos disparos y otros).

Usamos el método map de la biblioteca datasets y mostramos los resultados en una tabla de la siguiente manera:

In [ ]:
# let's predict over the test set to eval our zero-shot classifier
test_ds_with_preds = dataset["test"].select(range(1000)).map(lambda example: classify(example["text"]))

pd.set_option('display.max_colwidth', None)
test_ds_with_preds.to_pandas().head(15)

### Evaluando el clasificador


In [ ]:
# estas son las labels del dataset original
dataset["test"].features["label"].names

Ahora simplemente creamos el dataset en Argilla con las predicciones y las etiquetas del dataset original.

In [ ]:
import argilla as rg

# alineamos las labels con las que devuelve la API de OpenAI
labels = ['negative', 'neutral', 'positive']

records = []
for example in test_ds_with_preds:
    # create a record with ground-truth annotations and gpt-3 predictions
    record = rg.TextClassificationRecord(
        inputs={"text": example["text"], "explanation": example["explanation"]},
        annotation=labels[example["label"]],
        prediction=[(example["prediction"].lower(), 1.0)],
        metadata={"lang": example["lang"]}
    )
    records.append(record)

# create a dataset in Argilla
rg.log(records, "sentimiento_zs_gpt3")

### Métricas

Usando el módulo `metrics` de Argilla podemos analizar calidad de las predicciones:


In [ ]:
f1("sentimiento-zs-gpt3").visualize()

Y la distribución por longitud de texto:

In [ ]:
text_length("sentimiento-zs-gpt3").visualize()

Podemos analizar las métricas con respecto a la longitud del texto.

In [ ]:
f1("sentimiento-zs-gpt3", query="metrics.text_length:[0 TO 150]").visualize()

In [ ]:
f1("sentimiento-zs-gpt3", query="metrics.text_length:[150 TO *]").visualize()

## Clasificador zero-shot con SetFit


In [ ]:
labels = ["positivo", "negativo", "neutro"]

train_dataset = get_templated_dataset(
    candidate_labels=labels,
    sample_size=8,
    template="El tweet expresa un sentimiento {}"
)

model = SetFitModel.from_pretrained("hackathon-pln-es/paraphrase-spanish-distilroberta")
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset
)
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 960
  Num epochs = 1
  Total optimization steps = 60
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

### Evaluando el clasificador


In [ ]:
ds =  dataset["test"].select(range(1000))

def get_predictions(texts):
    mapping = {"negativo": "negative", "positivo": "positive", "neutro": "neutral"}
    probas = model.predict_proba(texts, as_numpy=True)
    for pred in probas:
        yield [{"label": mapping[label], "score": score} for label, score in zip(labels, pred)]

def get_annotation(labels):
  mapping = ['negative', 'neutral', 'positive']
  return [mapping[l] for l in labels]


dataset2 = ds.map(
    lambda batch: {
        "prediction": list(get_predictions(batch["text"])),
        "annotation": get_annotation(batch["label"])
    }, 
    batched=True,
    batch_size=10
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
rg_ds = rg.DatasetForTextClassification.from_datasets(dataset2)

rg.log(
    name="sentimiento-zs-setfit",
    records=rg_ds
)

Output()

1000 records logged to https://dvilasuero-taller-somosnlp.hf.space/datasets/team/sentimiento-zs-setfit

BulkResponse(dataset='sentimiento-zs-setfit', processed=1000, failed=0)

### Métricas

In [ ]:
f1("sentimiento-zs-setfit").visualize()

In [ ]:
f1("sentimiento-zs-setfit", query="score:[0.5 TO *]").visualize()

## Clasificador few-shot con SetFit

In [ ]:
small_ds = dataset["train"].shuffle()

In [ ]:
# Use cuda if available, if not, change cpu
encoder = SentenceTransformer("hackathon-pln-es/paraphrase-spanish-distilroberta", device="cuda")

# Encode text field using batched computation
small_ds = small_ds.map(
    lambda batch: {"vectors": encoder.encode(batch["text"])},
    batch_size=32,
    batched=True
)

# Removes the original labels because you'll be labelling from scratch
small_ds = small_ds.remove_columns("label")

# Turn vectors into a dictionary
small_ds = small_ds.map(
    lambda r: {"vectors": {"tweet_embedding": r["vectors"]}}
)

In [ ]:
small_ds

In [ ]:
settings = rg.TextClassificationSettings(
    label_schema=["positive", "negative", "neutral"]
)
rg.configure_dataset(name="tweets_for_labelling", settings=settings)

In [ ]:
rg_ds = rg.DatasetForTextClassification.from_datasets(small_ds, metadata="lang")

rg.log(
    name="tweets_for_labelling",
    records=rg_ds,
    chunk_size=100,
)

### Etiquetar!



In [ ]:
%%html
<iframe
	src="https://dvilasuero-taller-somosnlp.hf.space"
	frameborder="0"
	width="100%"
	height="700"
></iframe>

### Entrenar SetFit

In [ ]:
# Load the hand-labelled dataset from Argilla
ds = rg.load("tweets_for_labelling").prepare_for_training(train_size=0.8)

print(ds)

# Load SetFit model from Hub
model = SetFitModel.from_pretrained("hackathon-pln-es/paraphrase-spanish-distilroberta")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    loss_class=CosineSimilarityLoss,
    batch_size=8,
    num_iterations=20,
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate()
print(metrics)

### Evaluar

In [ ]:
ds =  dataset["test"].select(range(1000))

def get_predictions(texts):
    labels = ['negative', 'neutral', 'positive']
    probas = model.predict_proba(texts, as_numpy=True)
    for pred in probas:
        yield [{"label": label, "score": score} for label, score in zip(labels, pred)]

def get_annotation(labels):
  mapping = ['negative', 'neutral', 'positive']
  return [mapping[l] for l in labels]


dataset2 = ds.map(
    lambda batch: {
        "prediction": list(get_predictions(batch["text"])),
        "annotation": get_annotation(batch["label"])
    }, 
    batched=True,
    batch_size=10
)

In [ ]:
rg_ds = rg.DatasetForTextClassification.from_datasets(dataset2)

rg.log(
    name="sentimiento-fewshot-setfit",
    records=rg_ds
)

In [ ]:
f1("sentimiento-fewshot-setfit").visualize()

In [ ]:
f1("sentimiento-zs-setfit", query="score:[0.6 TO *]").visualize()

## Siguientes pasos

* Participa en el Hackathon creando tu propio dataset o al menos evaluando tus modelos con un dataset etiquetado. Por mucha excitación que haya con todos los nuevos modelos, la calidad de los datos y la verificación humana sigue siendo clave!

* Únete a la comunidad de Slack de Argilla y comparte tus preguntas, feedback y casos de uso!

* Si quieres apoyar el proyecto, contribuye y dejanos una estrella en GitHub: https://github.com/argilla-io/argilla

* El jueves Natalia Elvira estará resolviendo dudas y preguntas sobre proyectos de anotación como parte del hackathon, no te lo pierdas!



## Ideas de mejora



Para GPT models:

1. Hacer few-shot con `text-davinci-003` añadiendo ejemplos en el siguiente prompt:


In [ ]:
PROMPT_TEMPLATE = """
Classify the sentiment of the tweet in Spanish using the following JSON format. Use positive, negative, and neutral in lowercase:

{"prediction": sentiment label string, "explanation": sentence string describing in Spanish why you think is the sentiment}

Tweet:

"""

2. Usar `gpt3.5-turbo`, para ello hay que utilizar el endpoint chat
3. Usar `langchain` y/o `guardrails` para estructurar mejor el input y output del modelo.


Para SetFit:

1. Usar active learning para etiquetar más ejemplos, la librería `small-text` soporta modelos setfit.